In [7]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from pprint import pprint as pp

In [8]:
df_us = pickle.load(open('../pickles/df_c19us.p', 'rb'))
df_us_state = pickle.load(open('../pickles/df_c19us_state.p', 'rb'))
df_us_county = pickle.load(open('../pickles/df_c19us_county.p', 'rb'))
df = df_us_county['cases']